In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from tqdm import tqdm
from osgeo import gdal
import os

In [ ]:
print("os")
# Lists all items in the current directory
directories = os.listdir()
print(directories)

# Filter to find a specific folder
target_directory = next((dir for dir in directories if "coastalSentinel" in dir), None)

if target_directory:
    # Change working directory
    os.chdir(target_directory)
    print(f"Path changed to: {os.getcwd()}")
    # If you then want to go to the 'src' subfolder
    if 'src' in os.listdir():
        os.chdir('src')
        print(f"Path changed to subfolder 'src': {os.getcwd()}")
    else:
        print("'src' subfolder does not exist.")
else:
    print("Directory 'coastalSentinel' not find.")
    # If you then want to go to the 'src' subfolder
    if 'src' in os.listdir():
        os.chdir('src')
        print(f"Path changed to subfolder 'src': {os.getcwd()}")
    else:
        print("'src' subfolder does not exist.")

In [ ]:

# Function to load and preprocess images
def load_test(path):
    img = gdal.Open(path).ReadAsArray()
    stack_img = np.stack(img, axis=-1)
    rgb_img = get_rgb(stack_img)
    label_path = path.replace("images", "labels").replace("image", "label")
    label = gdal.Open(label_path).ReadAsArray() if os.path.exists(label_path) else None
    return stack_img, rgb_img, label

def get_rgb(img):
    """Return normalized RGB channels from sentinel image"""
    rgb_img = img[:, :, [3,2,1]]
    rgb_normalize = np.clip(rgb_img/10000, 0, 0.3)/0.3
    return (rgb_normalize * 255).astype(np.uint8)  # Convert to uint8

# Load images and labels
def load_data():
    test_path = glob.glob("../data/SWED/test/images/*")
    print(test_path[0])
    print(len(test_path))

    input_images = []
    rgb_images = []
    labels = []
    image_names = []

    for path in tqdm(test_path, desc="Loading data"):
        try:
            img, rgb_img, label = load_test(path)
            input_images.append(img)
            rgb_images.append(rgb_img)
            labels.append(label)
            image_names.append(os.path.basename(path))  # Store image name
        except Exception as e:
            print(f"Error with image {path}: {e}")

    return rgb_images, labels, image_names

# Function to generate label image where blue water becomes white and land becomes black
def generate_label(image):
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Define multiple color ranges for detecting different shades of blue
    lower_blue1 = np.array([90, 50, 50])
    upper_blue1 = np.array([130, 255, 255])
    
    lower_blue2 = np.array([90, 50, 50])
    upper_blue2 = np.array([110, 255, 255])
    
    lower_blue3 = np.array([100, 50, 50])
    upper_blue3 = np.array([140, 255, 255])
    
    # Create masks for the different blue ranges
    mask_blue1 = cv2.inRange(hsv_image, lower_blue1, upper_blue1)
    mask_blue2 = cv2.inRange(hsv_image, lower_blue2, upper_blue2)
    mask_blue3 = cv2.inRange(hsv_image, lower_blue3, upper_blue3)
    
    # Combine the masks
    mask_water = cv2.bitwise_or(mask_blue1, mask_blue2)
    mask_water = cv2.bitwise_or(mask_water, mask_blue3)

    # Additional step: Use Canny edge detection to help with water boundary detection
    edges = cv2.Canny(image, 100, 200)
    
    # Combine mask and edges
    mask_combined = cv2.bitwise_or(mask_water, edges)
    
    # Initialize the label image with the same dimensions as the input image
    label_image = np.zeros(image.shape[:2], dtype=np.uint8)
    
    # Set water regions to white (255)
    label_image[mask_combined > 0] = 255
    
    return label_image

In [ ]:

# Load and preprocess the data
rgb_images, labels, image_names = load_data()

# Create output directory if it doesn't exist
output_dir = "../data/SWED/reference_image_generate/"
os.makedirs(output_dir, exist_ok=True)

# Process each image and save the generated label
for rgb_image, image_name in zip(rgb_images, image_names):
    label_image = generate_label(rgb_image)
    output_path = os.path.join(output_dir, image_name.replace(".png", "_reference_generate.png"))
    cv2.imwrite(output_path, label_image)

    # Display the original and label images
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(rgb_image)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title('reference generate Image')
    plt.imshow(label_image, cmap='gray')
    plt.axis('off')

    plt.show()

#Now i want to see what i can do with hawai images


In [ ]:
# Define the paths to the images
raw_image_path = '../hawai/huawaiaugust2014.png'
annotated_image_path = '../hawai/annotated/huawaiaugust2014_annotated.png'

# Load the raw and annotated images
raw_image = cv2.imread(raw_image_path)
annotated_image = cv2.imread(annotated_image_path)

# Display the images for verification
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Raw Image')
plt.imshow(cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Annotated Image')
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.show()